In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import matplotlib.pyplot as plt
import os, sys
sns.set()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:
df = pd.read_csv("./UFO Sightings Analysis/ufo-sightings-transformed.csv").drop(columns=["Unnamed: 0"])
df.head(10)

,Date_time,date_documented,Year,Month,Hour,Season,Country_Code,Country,Region,Locale,latitude,longitude,UFO_shape,length_of_encounter_seconds,Encounter_Duration,Description
0,1949-10-10 20:30:00,4/27/2004,1949,10,20,Autumn,USA,United States,Texas,San Marcos,29.883056,-97.941111,Cylinder,2700.0,45 minutes,This event took place in early fall around 194...
1,1949-10-10 21:00:00,12/16/2005,1949,10,21,Autumn,USA,United States,Texas,Bexar County,29.384210,-98.581082,Light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...
2,1955-10-10 17:00:00,1/21/2008,1955,10,17,Autumn,GBR,United Kingdom,England,Chester,53.200000,-2.916667,Circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...
3,1956-10-10 21:00:00,1/17/2004,1956,10,21,Autumn,USA,United States,Texas,Edna,28.978333,-96.645833,Circle,20.0,1/2 hour,My older brother and twin sister were leaving ...
4,1960-10-10 20:00:00,1/22/2004,1960,10,20,Autumn,USA,United States,Hawaii,Kaneohe,21.418056,-157.803611,Light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...
5,1961-10-10 19:00:00,4/27/2007,1961,10,19,Autumn,USA,United States,Tennessee,Bristol,36.595000,-82.188889,Sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...
6,1965-10-10 21:00:00,2/14/2006,1965,10,21,Autumn,GBR,United Kingdom,Wales,Penarth,51.434722,-3.180000,Circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...
7,1965-10-10 23:45:00,10/2/1999,1965,10,23,Autumn,USA,United States,Connecticut,Norwalk,41.117500,-73.408333,Disk,1200.0,20 minutes,A bright orange color changing to reddish colo...
8,1966-10-10 20:00:00,3/19/2009,1966,10,20,Autumn,USA,United States,Alabama,Harrisburg,33.586111,-86.286111,Disk,180.0,3 minutes,Strobe Lighted disk shape object observed clos...
9,1966-10-10 21:00:00,5/11/2005,1966,10,21,Autumn,USA,United States,Florida,Live Oak,30.294722,-82.984167,Disk,120.0,several minutes,Saucer zaps energy from powerline as my pregna...


In [13]:
os.getcwd()

'/Users/cuji/Programming Projects'